# An Evaluation of Precipitation Forecast Product for Dynamic Wetlands Management

In [227]:
from siphon.catalog import TDSCatalog
top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
for ref in top_cat.catalog_refs:
    print(ref)

Forecast Model Data
Forecast Products and Analyses
Observation Data
Radar Data
Satellite Data
Unidata case studies


In [219]:
import xarray as xr
import numpy as np
from datetime import datetime, timedelta
import pyproj
from pyproj import Proj
import os
from scipy import interpolate
import rasterio
from rasterio.transform import from_origin


def to_datetime(date):
    """
    Converts a numpy datetime64 object to a python datetime object 
    Input:
      date - a np.datetime64 object
    Output:
      DATE - a python datetime object
    """
    timestamp = ((date - np.datetime64('1970-01-01T00:00:00'))
                 / np.timedelta64(1, 's'))
    return datetime.utcfromtimestamp(timestamp)

ds = xr.open_dataset('TIGGE/201207_PerturbedAll.grib', engine="cfgrib")

#separate variables
lon = np.asarray(ds['longitude'])
lat = np.asarray(ds['latitude'])
time = np.asarray(ds['time'])
data = ds['tp']
EnsNumber = ds['number']
LeadTimes = ds['step']

#  # Convert the number of hours since the reference time to an actual date, check the array index
#     time_val = num2date(time_var[timeId].squeeze(), time_var.units)
#     print(num2date(time_var[timeId].squeeze(), time_var.units))

#projection
GRIB2_proj = Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs") #put pm=-360 to indicate GRIB2 projection
Projected = Proj(init="EPSG:26914") #26914 Cypress UTM 14N

#resample dx
res = 1000.

#convert coordinate
x,y = np.meshgrid(lon, lat)
UTMx, UTMy = pyproj.transform(GRIB2_proj, Projected, x.flatten(), y.flatten())
UTMx_grid = np.reshape(UTMx,(x.shape))
UTMy_grid = np.reshape(UTMy,(y.shape))
xmin, xmax, ymin, ymax = [UTMx.min(), UTMx.max(), UTMy.min(), UTMy.max()]
UTMx_res = np.arange(xmin, xmax, res)
UTMy_res = np.arange(ymin, ymax, res)
grid_x_res, grid_y_res = np.meshgrid(UTMx_res, UTMy_res)
nrows_res, ncols_res = np.shape(grid_x_res)

#select variables : 'tp' (number: 20, time: 124, step: 9, latitude: 7, longitude: 5)
LeadTime = 0
EnsMember = 0
t = 0


save_dir = 'Precipitation_Cypress_GEFS/'
# save_dir_asc = save_dir + dirASC

for EnsMember in range(EnsNumber.size):
    #create folder for outputs files
    for LeadTime in range(LeadTimes.size):
        save_dir1 = save_dir + str(EnsMember) + '/' + str(LeadTime) + '/'
        if not os.path.isdir(save_dir1):
            os.makedirs(save_dir1)
        
        for t in range(time.size):
            ds_subset_date = to_datetime(time[t])
            ds_datetime = ds_subset_date.strftime("%Y%m%d%H")
            filenameASC = save_dir1 + 'GEFS.ens'+ str(EnsMember) + '.f' + str(LeadTime)+ '.' + ds_datetime + '.asc'

            Precipitation_vals = np.asarray(data[dict(number=EnsMember, step=LeadTime, time=t)])
            Precipitation_res = interpolate.griddata((UTMx_grid.flatten(), UTMy_grid.flatten()), Precipitation_vals.flatten(), (grid_x_res, grid_y_res) , method='cubic')
            Precipitation_res[np.isnan(Precipitation_res)] = -9999

            #save resampled raster to ascii
            transform = from_origin(grid_x_res.min(), grid_y_res.max(), res, res)
            new_dataset = rasterio.open(filenameASC, 'w', nodata = -9999, driver='AAIGrid', decimal_precision=1,
                                        height = nrows_res, width = ncols_res,
                                        count=1, dtype=str(Precipitation_res.dtype),
                                        transform=transform,
                                        crs=Projected.srs)
            new_dataset.write(Precipitation_res, 1)
            new_dataset.close()

## Convert GEFS to DSS

In [1]:
# convert to DSS
import re
import os
import glob 
import platform
from datetime import datetime, timedelta

# # Local variables:
# my_path = os.path.abspath(os.path.dirname(__file__))
# inDir = os.path.join(my_path, "..\PrecForecast\GFS\ASCII")
# openBATfile = my_path  + "\Batch_ASCIIToDSS.bat"

# Local variables:
#inDir = 'Precipitation_Mongolia/ASC_2000_Dis/'
save_dir = 'Precipitation_Cypress_GEFS/'


for EnsMember in range(0, 20): 
    for LeadTime in range(2, 6):
    #LeadTime = 3
        save_dir1 = save_dir + str(EnsMember) + '/' + str(LeadTime) + '/'
        openBATfile = save_dir1 + 'ASCIIToDSS.bat'
        DSSNAME = "GEFS-Ens-" + str(EnsMember) + "-f-" + str(LeadTime)
        # for generating asc2dss strings
        text_1 = "asc2dssGrid INPUT="
        text_2 = " DSS=" + DSSNAME + ".dss PATH=/UTM14/Cypress/Precip/"
        text_3 = "/"
        text_4 = "/PROJECTED/ GRIDTYPE=UTM ZONE=14N DUNITS=mm DTYPE=PER-CUM" 

        # Open and Write to file1  
        file1 = open(openBATfile,"w")

        date = [date for file in glob.glob(save_dir1+'*.asc') for date in re.findall("(\d{10})", file)]
        date.sort(key = lambda date: datetime.strptime(date, "%Y%m%d%H")) 
        delt = datetime.strptime(date[1], "%Y%m%d%H") - datetime.strptime(date[0], "%Y%m%d%H")

        AscFiles = glob.glob(save_dir1+'*.asc')


        for index in range(0, len(date)):
        # +'gefs.1deg.ens.'+ str(EnsMember)+ '.' + time_val.strftime("%Y%m%d%H") + '.asc'
            namefile = 'GEFS.ens'+ str(EnsMember) + '.' + 'f' + str(LeadTime) + '.' +date[index] + '.asc'
            objDate = datetime.strptime(date[index], '%Y%m%d%H')

            if (objDate.hour==0): 
                objDate0 = objDate - delt #timedelta(hours=1)
                objDate = objDate - delt #timedelta(days=1)
                converted_date = datetime.strftime(objDate,'%d%b%Y:2400') #HEC-DSS only accept 24:00 format!
            else:
                converted_date = datetime.strftime(objDate,'%d%b%Y:%H%M')
                objDate0 = objDate - delt#timedelta(hours=1)

            converted_date0 = datetime.strftime(objDate0,'%d%b%Y:%H%M')

            full_text = text_1 + namefile + text_2 + converted_date0 + text_3 + converted_date + text_4
            file1.write(full_text + "\n")
        
        
        file1.close() #to close file   
        
        os.chdir(save_dir1)
        # convert asc to DSS
        os.startfile('ASCIIToDSS.bat')
#         # Move DSS files to HEC-HMS Working directory
#         shutil.copytree(src, dst)
        os.chdir('C:\\temp\\Flood_Forecasting')
        #os.getcwd()

In [10]:
# Move DSS files to HEC-HMS Working directory
import shutil
import glob, os

src = 'C:\\temp\\Flood_Forecasting\\Precipitation_Cypress_GEFS_2012'
dst = 'C:\\temp\\Flood_Forecasting\\CypressHMS1\\Cypress_2012\\forecast'
os.chdir(src)

for root, dirs, files in os.walk(src):
    for file in files:
        if file.endswith(".dss"):
            src_temp = os.path.join(root, file)
            shutil.copy(src_temp, dst)

## MSWEP Precipitation dataset

In [314]:
# open dataset
mswep = xr.open_dataset('Precipitation_MSWEP/201207.nc')
time = np.asarray(mswep['time'])

#projection
GRIB2_proj = Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs") #put pm=-360 to indicate GRIB2 projection
Projected = Proj(init="EPSG:26914") #26914 Cypress UTM 14N

#resample dx
res = 1000.

# subset dataset
West = -97
East = -93.5
North = 31.5
South = 29.5
Precipitation_mswep = mswep.sel(lon=[West, East], lat=[South, North], method='nearest').to_array()[0]
lon=Precipitation_mswep['lon']
lat=Precipitation_mswep['lat']

#convert coordinate
x,y = np.meshgrid(lon, lat)
UTMx, UTMy = pyproj.transform(GRIB2_proj, Projected, x.flatten(), y.flatten())
UTMx_grid = np.reshape(UTMx,(x.shape))
UTMy_grid = np.reshape(UTMy,(y.shape))
xmin, xmax, ymin, ymax = [UTMx.min(), UTMx.max(), UTMy.min(), UTMy.max()]
UTMx_res = np.arange(xmin, xmax, res)
UTMy_res = np.arange(ymin, ymax, res)
grid_x_res, grid_y_res = np.meshgrid(UTMx_res, UTMy_res)
nrows_res, ncols_res = np.shape(grid_x_res)


save_dir = 'Precipitation_Cypress_MSWEP/'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

    
# loop through time
for t in range(time.size):
    ds_subset_date = to_datetime(time[t])
    ds_datetime = ds_subset_date.strftime("%Y%m%d%H")
    filenameASC = save_dir + 'MSWEP' + '.' + ds_datetime + '.asc'
    
    Precipitation_vals = np.asarray(Precipitation_mswep[:,:,t])  
    Precipitation_res = interpolate.griddata((UTMx_grid.flatten(), UTMy_grid.flatten()), 
                                             Precipitation_vals.flatten(), (grid_x_res, grid_y_res), 
                                             method='cubic')
    Precipitation_res[np.isnan(Precipitation_res)] = -9999

    #save resampled raster to ascii
    transform = from_origin(grid_x_res.min(), grid_y_res.max(), res, res)
    new_dataset = rasterio.open(filenameASC, 'w', nodata = -9999, driver='AAIGrid', decimal_precision=1,
                                height = nrows_res, width = ncols_res,
                                count=1, dtype=str(Precipitation_res.dtype),
                                transform=transform,
                                crs=Projected.srs)
    new_dataset.write(Precipitation_res, 1)
    new_dataset.close()

## Convert MSWEPv1 to DSS

In [1]:
# convert to DSS
import re
import glob 
import platform
from datetime import datetime, timedelta

# Local variables:
# my_path = os.path.abspath(os.path.dirname(__file__))
# inDir = os.path.join(my_path, "..\Forecast_GFS")
# openBATfile = my_path  + "\ASCIIToDSS.bat"

# Local variables:
#inDir = 'Precipitation_Mongolia/ASC_2000_Dis/'
save_dir = 'Precipitation_Cypress_MSWEP/'
openBATfile = save_dir + 'ASCIIToDSS.bat'

# for generating asc2dss strings
text_1 = "asc2dssGrid INPUT="
text_2 = " DSS=MSWEP.dss PATH=/UTM14/Cypress/Precip/"
text_3 = "/"
text_4 = "/PROJECTED/ GRIDTYPE=UTM ZONE=14N DUNITS=mm DTYPE=PER-CUM" 

# Open and Write to file1  
file1 = open(openBATfile,"w")

# date = [date for file in glob.glob(inDir + '*.asc') for date in re.findall("(\d{10})", file)]
date = [date for file in glob.glob(save_dir + '*.asc') for date in re.findall("(\d{10})", file)]
date.sort(key = lambda date: datetime.strptime(date, "%Y%m%d%H")) 
delt = datetime.strptime(date[1], "%Y%m%d%H") - datetime.strptime(date[0], "%Y%m%d%H")
# AscFiles = glob.glob(inDir + '*.asc')
AscFiles = glob.glob(save_dir + '*.asc')

for index in range(0, len(date)):

    namefile = 'MSWEP.' + date[index] + '.asc'
    objDate = datetime.strptime(date[index], '%Y%m%d%H')

    if (objDate.hour==0): 
        objDate0 = objDate - delt #timedelta(hours=1)
        objDate = objDate - delt #timedelta(days=1)
        converted_date = datetime.strftime(objDate,'%d%b%Y:2400') #HEC-DSS only accept 24:00 format!
    else:
        converted_date = datetime.strftime(objDate,'%d%b%Y:%H%M')
        objDate0 = objDate - delt#timedelta(hours=1)
    
    converted_date0 = datetime.strftime(objDate0,'%d%b%Y:%H%M')

    full_text = text_1 + namefile + text_2 + converted_date0 + text_3 + converted_date + text_4
    file1.write(full_text + "\n")
    
file1.close() #to close file   